<a href="https://colab.research.google.com/github/Gandhravs/Gandhrav_Sharma_AER850_Project3/blob/main/AER_850_PROJECT_3_FINAL_TRY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# starting on friday 12/13/2024

In [ ]:
# Step 1: Object Masking with Contour Extraction

# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define the path to the input image
image_path = "/content/drive/MyDrive/AER_850_DATA/data/motherboard_image.JPEG"

# Load and preprocess the image
original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)

# Apply Gaussian blur to smoothen the image
blurred_image = cv2.GaussianBlur(original_image, (47, 47), 4)

# Convert to grayscale for further processing
gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)
rotated_gray_image = cv2.rotate(gray_image, cv2.ROTATE_90_CLOCKWISE)

# Perform adaptive thresholding to segment the image
threshold_image = cv2.adaptiveThreshold(
    rotated_gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 7)

# Detect edges using the Canny edge detector
edges_detected = cv2.Canny(threshold_image, 50, 350)
dilated_edges = cv2.dilate(edges_detected, None, iterations=12)

# Identify contours from the edges
detected_contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Prepare a blank mask to draw the largest contour
contour_mask = np.zeros_like(rotated_image)

# Find and draw the largest contour
largest_contour = max(detected_contours, key=cv2.contourArea)
cv2.drawContours(
    image=contour_mask, contours=[largest_contour], contourIdx=-1, color=(255, 255, 255), thickness=cv2.FILLED
)

# Apply the mask to the original rotated image

final_masked_image = cv2.bitwise_and(contour_mask, rotated_image)

# change to rgb
edges_rgb = cv2.cvtColor(edges_detected, cv2.COLOR_BGR2RGB)
mask_rgb = cv2.cvtColor(contour_mask, cv2.COLOR_BGR2RGB)
final_masked_rgb = cv2.cvtColor(final_masked_image, cv2.COLOR_BGR2RGB)


# Display the result

plt.figure(figsize=(20, 5))
plt.subplot(141)
plt.imshow(edges_rgb)
plt.title('Edge Detection')
plt.axis('off')

plt.subplot(143)
plt.imshow(mask_rgb)
plt.title('Masked Image')
plt.axis('off')

plt.subplot(144)
plt.imshow(final_masked_rgb)
plt.title('Final Extracted PCB')
plt.axis('off')

plt.tight_layout()
plt.show()

In [1]:
# Step 2 Yolo V8 install

!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 20.9 MB/s eta 0:00:00


In [2]:
# Step 2 Yolo V8

from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

model.train(data="/content/drive/MyDrive/Project_Data/data/data.yaml" , epochs=175, imgsz = 1200, batch = 3, name = 'yolo_model_project_rasp')  # train the model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 126MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Project_Data/data/data.yaml, epochs=175, time=None, patience=100, batch=3, imgsz=1200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo_model_project_rasp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

100%|██████████| 755k/755k [00:00<00:00, 21.8MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

    175/175      8.02G      1.406     0.8813     0.9187         38       1216: 100%|██████████| 182/182 [00:52<00:00,  3.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:04<00:00,  3.74it/s]


                   all        105      19108      0.852      0.723       0.79      0.581

175 epochs completed in 2.994 hours.
Optimizer stripped from runs/detect/yolo_model_project_rasp/weights/last.pt, 6.4MB
Optimizer stripped from runs/detect/yolo_model_project_rasp/weights/best.pt, 6.4MB

Validating runs/detect/yolo_model_project_rasp/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,008,183 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:13<00:00,  1.33it/s]


                   all        105      19108      0.849      0.728      0.796       0.59
                Button         14         45      0.904          1      0.995      0.832
             Capacitor        104       7251      0.836      0.516      0.666       0.36
             Connector         76        659      0.853      0.915      0.923      0.725
                 Diode         11         53      0.739      0.453      0.526      0.404
Electrolytic Capacitor         15        160      0.929      0.963       0.98      0.747
                    IC         95       1322      0.894      0.933      0.952      0.744
              Inductor         12         45      0.937      0.663      0.808      0.628
                   Led         18        127      0.867      0.858      0.909      0.684
                  Pads         20        143      0.698      0.324      0.412      0.292
                  Pins         10        151      0.791      0.901       0.92      0.668
              Resisto

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e5a08c34340>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504